In [1]:
# 🔁 Import pipeline components
from regressorpipeline.train import train_fire_model, train_multiple_cnn_for_fire
from regressorpipeline.predict import (
    load_model_bundle,
    predict_fire_risk,
    predict_fire_risk_from_models,
)
from regressorpipeline.visualize import plot_fire_risk_surface_matplotlib

import pandas as pd
import os


In [5]:
# ✅ Test paths
train_path = "example_data_train.xlsx"
test_path = "example_data_test.xlsx"

from regressorpipeline.train import train_fire_model, train_multiple_cnn_for_fire
from regressorpipeline.predict import (
    load_model_bundle,
    predict_fire_risk,
    predict_fire_risk_from_models,
)
from regressorpipeline.visualize import plot_fire_risk_surface_matplotlib
import pandas as pd
import numpy as np

# 🔧 1. Train and evaluate a single CNN model
model, metrics = train_fire_model("cnn", train_path, save=True)
print("Single CNN model metrics:", metrics)

# 🔧 2. Train CNN ensemble (3 runs) and save
models, metrics_list, ensemble_metrics = train_multiple_cnn_for_fire(
    "example_data_train.xlsx", n_runs=3, save=True
)
print("Ensemble metrics:", ensemble_metrics)

# 🔍 3. Load and test prediction (single CNN model)
models, scaler_X, scaler_y = load_model_bundle("best_cnn_model.joblib")
preds = predict_fire_risk(models[0], scaler_X, scaler_y, "example_data_test.xlsx")
print("Single CNN Predictions:", preds[:5])

# 🔍 4. Load and test ensemble prediction
models, scaler_X, scaler_y = load_model_bundle("cnn_ensemble.joblib")
avg_preds = predict_fire_risk_from_models(models, scaler_X, scaler_y, "example_data_test.xlsx")
print("Averaged Ensemble Predictions:", avg_preds[:5])

# 🔍 5. Load and test OLS model prediction
models, scaler_X, scaler_y = load_model_bundle("best_ols_model.joblib")
ols_preds = predict_fire_risk(models[0], scaler_X, scaler_y, "example_data_test.xlsx")
print("OLS Predictions:", ols_preds[:5])

# 📊 6. Generate CNN surface plot
df = pd.read_excel(test_path, engine="openpyxl")
X_scaled_df = pd.DataFrame(
    scaler_X.transform(np.log1p(df.select_dtypes(include="number"))),
    columns=df.select_dtypes(include="number").columns
)

plot_fire_risk_surface_matplotlib(
    model=models[0],
    X_scaled_df=X_scaled_df,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    feat1_name="Thermal Inertia",  # must match feature name
    feat2_name="HRRPUA",           # must match feature name
    title="CNN Prediction Surface",
    save_path="examples/cnn_surface.html"
)
print("✅ Surface plot saved to examples/cnn_surface.html")



[I 2025-05-31 17:41:38,613] A new study created in memory with name: no-name-2f4ed193-84c7-4380-a58b-aa0c010fe4f1
[I 2025-05-31 17:41:38,713] Trial 0 finished with value: 1.5684123319593657 and parameters: {'num_filters1': 28, 'num_filters2': 60, 'fc1_size': 122, 'lr': 0.0003893169840519166}. Best is trial 0 with value: 1.5684123319593657.
[I 2025-05-31 17:41:38,745] Trial 1 finished with value: 13.233178078418334 and parameters: {'num_filters1': 13, 'num_filters2': 28, 'fc1_size': 122, 'lr': 0.00017235877571519914}. Best is trial 0 with value: 1.5684123319593657.
[I 2025-05-31 17:41:38,782] Trial 2 finished with value: 7.675171330909647 and parameters: {'num_filters1': 32, 'num_filters2': 31, 'fc1_size': 83, 'lr': 0.00017644409553244024}. Best is trial 0 with value: 1.5684123319593657.
[I 2025-05-31 17:41:38,814] Trial 3 finished with value: 0.4921625426233851 and parameters: {'num_filters1': 19, 'num_filters2': 29, 'fc1_size': 47, 'lr': 0.002070712245135045}. Best is trial 3 with val


🔥 Model 'cnn' Evaluation:
R²: 0.4025
MAE: 0.1028
MSE: 0.0205
Single CNN model metrics: {'R²': 0.4025452597331277, 'MAE': 0.102795094, 'MSE': 0.02045743}


[I 2025-05-31 17:41:39,266] Trial 5 finished with value: -0.5923202030407622 and parameters: {'num_filters1': 19, 'num_filters2': 51, 'fc1_size': 33, 'lr': 0.002294883487399579}. Best is trial 5 with value: -0.5923202030407622.
[I 2025-05-31 17:41:39,300] Trial 6 finished with value: 1.2255019017003304 and parameters: {'num_filters1': 25, 'num_filters2': 42, 'fc1_size': 76, 'lr': 0.0011397456120426253}. Best is trial 5 with value: -0.5923202030407622.
[I 2025-05-31 17:41:39,327] Trial 7 finished with value: 10.602710269551224 and parameters: {'num_filters1': 12, 'num_filters2': 42, 'fc1_size': 33, 'lr': 0.00014623220532424843}. Best is trial 5 with value: -0.5923202030407622.
[I 2025-05-31 17:41:39,359] Trial 8 finished with value: 7.149629505905393 and parameters: {'num_filters1': 30, 'num_filters2': 44, 'fc1_size': 82, 'lr': 0.0003122593363950876}. Best is trial 5 with value: -0.5923202030407622.
[I 2025-05-31 17:41:39,390] Trial 9 finished with value: 0.09657327471110921 and paramet


🔥 CNN Ensemble Evaluation:
R²: 0.3933
MAE: 0.1102
MSE: 0.0208
Ensemble metrics: {'R²': 0.39333711629576973, 'MAE': 0.11019679765658241, 'MSE': 0.020772726309821073}
Single CNN Predictions: [4861.7197   4278.63       18.240555   94.019936  139.47878 ]
Averaged Ensemble Predictions: [4861.7197   4278.63       18.240555   94.019936  139.47878 ]


ValueError: shapes (17,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)